In [50]:
import pandas as pd
from lib.connect_db import get_engine
from lib.print_error import print_error
from nltk.corpus import stopwords
import inflect
p = inflect.engine()

In [51]:
df_main = pd.read_csv("db/definitions.csv", names=["word","definition"], sep=":", index_col=None, keep_default_na=False, na_values=[""])
df = df_main[:1000]
df.to_csv("db/sample/sample.csv", sep=":", columns=["word","definition"], header=False, index=False, mode="w")
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
word          1000 non-null object
definition    1000 non-null object
dtypes: object(2)
memory usage: 15.7+ KB
None


#### find meanings for words present in definition columns of df and append the df, save to sample-1.csv, save non-existing to not_found.csv

In [52]:
trivial_words = []

def treat_word(row):
    global df
    index = row.name
    for word in row.definition.split():
        if not any(df["word"] == word):
            if any(df_main["word"] == word):
                df = df.append({"word":word, "definition": df_main["definition"]}, ignore_index=True)
            else:
                trivial_words.append(word)

df.apply(treat_word, axis=1)
print("df after appending - ")
print(df.info())

print("\nnumber of words without definition - ")
print(len(trivial_words))

df after appending - 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2150 entries, 0 to 2149
Data columns (total 2 columns):
word          2150 non-null object
definition    2150 non-null object
dtypes: object(2)
memory usage: 33.7+ KB
None

number of words without definition - 
706


In [53]:
df_trivial = pd.DataFrame({'word': trivial_words}, index=None)
df_trivial.to_csv("db/sample/not_found.csv", columns=["word"], header=False, index=False, mode="w")
df.to_csv("db/sample/sample-1.csv", columns=["word"], header=False, index=False, mode="w")

In [44]:
print(df_trivial.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 706 entries, 0 to 705
Data columns (total 1 columns):
word    706 non-null object
dtypes: object(1)
memory usage: 5.6+ KB
None


#### convert plural to singular, search in definitons, if exists, save to sample-2.csv, save non existent to not-found.csv

In [45]:
def convert_to_singular(word):
    if p.singular_noun(word) != False:
        return p.singular_noun(word)
    return word

In [49]:
df_trivial = pd.read_csv("db/sample/not_found.csv", names=["word"], index_col=None, keep_default_na=False, na_values=[""])
sample_1 = pd.read_csv("db/sample/sample-1.csv", names=["word" "definition"], index_col=None, keep_default_na=False, na_values=[""])
definitions = pd.read_csv("db/sample/definitions.csv", names=["word", "definition"], index_col=None, keep_default_na=False, na_values=[""])

def treat_plurals(word):
    global df
    singular_word = p.singular(word)
    if singular_word != False:
        if any(df_main["word"] == word):
            df = df.append({"word": word, "definition": df_main["definition"]})

# df_non_trivial = pd.DataFrame(columns=["word"], index=None)
# df_non_trivial_final = pd.DataFrame(columns=["word", "definition"], index=None)

def filter_converted(word):
    global df_non_trivial
    if word != False:
        df_non_trivial = df_non_trivial.append({"word": word}, ignore_index=True)
    return

df_trivial["word"].apply(filter_converted)
df_non_trivial = df_non_trivial.drop_duplicates(keep = 'first')

def treat_word2(row):
    index = row.name
    word = row["word"]
    global df_non_trivial_final
    global df_trivial
    if any(df_main["word"] == word):
        df_non_trivial_final = df_non_trivial_final.append({"word":word, "definition": df_main["definition"]}, ignore_index=True)
        df_trivial = df_trivial.drop(index)
    
df_non_trivial.apply(treat_word2, axis=1)
print("found words df - ")
print(df_non_trivial_final.info())

df_trivial = df_trivial.reset_index()
df_trivial = df_trivial.drop_duplicates(keep = 'first')
print("\nnot found words df - ")
print(df_trivial.info())
df_non_trivial.to_csv("db/sample/sample-2.csv", columns=["word"], header=False, index=False, mode="w")    
df_trivial.to_csv("db/sample/not-found.csv", columns=["word"], header=False, index=False, mode="w")      

found words df - 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138 entries, 0 to 137
Data columns (total 2 columns):
word          138 non-null object
definition    138 non-null object
dtypes: object(2)
memory usage: 2.2+ KB
None

not found words df - 
<class 'pandas.core.frame.DataFrame'>
Int64Index: 292 entries, 0 to 704
Data columns (total 1 columns):
word    292 non-null object
dtypes: object(1)
memory usage: 4.6+ KB
None


In [ ]:
df_main = pd.read_csv("db/definitions.csv", names=["word","definition"], sep=":", index_col=None, keep_default_na=False, na_values=[""])
df_trivial = pd.read_csv("db/sample/sample-1.csv", names=["word"], index_col=None, keep_default_na=False, na_values=[""])

df_non_trivial = pd.DataFrame(columns=["word"], index=None)

def add_plural(word):
    global df_non_trivial
    plural_word = p.singular_noun(word)
    if plural_word != False:
        
    return

df_trivial.apply(add_plural)